Abbreviations of morphological classes used in folder structure and annotation file
----------------------------------------------------------------------------------------
**BAS** Basophil

**EBO** Erythroblast (and Monoblast)

**EOS** Eosinophil

**LYT** Lymphocyte (typical and atypical)

**MON** Monocyte

**MYO** Myeloblast (Metamyelocyte and Myelocyte)

**NGB** Neutrophil (band)

**NGS** Neutrophil (segmented)

**PMO** Promyelocyte (bilobled and not)

![Hematopoiesis_(human)_diagram_ru.svg.png](attachment:Hematopoiesis_(human)_diagram_ru.svg.png)

**Подготовка набора данных**

In [1]:
import os
import shutil
from dataset_preparing import create_labels, tiff_to_png 

In [3]:
labels = ['BAS', 'EBO', 'MON', 'LYT', 'EOS', 'MYO', 'NGB', 'NGS', 'PMO']
cur_path = os.getcwd()

In [ ]:
# Конвертируем .tiff в .png
tiff_to_png(path=f'{cur_path}\images_tiff', out=f'{cur_path}\images_png\\')

In [4]:
# Разделяем test и train выборки по соответствующим директориям
# 80% на train и 20% на test

for folder, _, files in os.walk(f'{cur_path}\images_png'):
    file_counter = 0
    for file in files:
        # Делим набор данных
        if file_counter % 5 == 0:
            shutil.copy(src=os.path.join(folder, file),
                        dst=f'dataset\\test\{file}')
        else:
            shutil.copy(src=os.path.join(folder, file),
                        dst=f'dataset\\train\{file}')
        file_counter+=1

In [4]:
# Создаём метки классов в формате csv для каждого изображения в test и train 

train_path = f'{cur_path}\\dataset\\train'
test_path = f'{cur_path}\\dataset\\test'

create_labels(path=train_path, csv_path=train_path + '\\train.csv')
create_labels(path=test_path, csv_path=test_path+ '\\test.csv')

In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
from dataloader import LeukocytesDataset, Rescale, RandomCrop, ToTensor

In [2]:
# Define relevant variables for the ML task
batch_size = 4
num_classes = 9
learning_rate = 0.001
num_epochs = 20

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([dl.Resize((32,32)),
                                     dl.ToTensor()
                                     ])

train_dataset = dl.LeukocytesDataset(root_dir='dataset/train',
                                    csv_file='dataset/labels.csv',
                                    )